## importing pad_sequences,to_categorical

In [1]:
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import numpy as np
###read from step1,2

## Reading   vocab , training data and augmanted data

In [1]:
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import numpy as np###read from step1,2
vocabulary=np.load('./vocabulary (1).npy')
##output of CNN without the prediction layer or any additional FC layers size(2048,0)
#images_features=np.load('./training_features_after_pooling.npy')
images_features=np.load('./augmented_training_features_after_pooling.npy')
#images_features = np.concatenate((images_features_1, images_features_2), axis=0)
#read from tokeniszed captions
captions_dict=np.load('./training_tokenized_captions.npy')
#captions_dict = np.concatenate((captions_dict, captions_dict), axis=0)
print(vocabulary)
print(images_features.shape,len(images_features))
print(len(captions_dict))

[['a' '1']
 ['.' '2']
 ['in' '3']
 ...
 ['non-working' '2727']
 ['startseq' '2728']
 ['endseq' '2729']]
(3000, 2048) 3000
3000


In [2]:
print(images_features.shape)
print(captions_dict.shape)

(3000, 2048)
(3000, 37)


# generation of data passed to the model for training 

In [3]:
VOCAB_SIZE = len(vocabulary)
MAX_LEN=37
def generator(images, caption):
    n_samples = 0

    X = []
    y_in = []
    y_out = []

    for i in range(0,len(images_features)):
        for j in range (1,len(captions_dict[i])-1):

            X.append(images_features[i])
            in_seq= [captions_dict[i][:j]]
            out_seq = captions_dict[i][j]
            in_seq = pad_sequences(in_seq, maxlen=MAX_LEN, padding='post', truncating='post')[0]
            # out_seq = to_categorical(int(out_seq), num_classes=VOCAB_SIZE)[0]
            out = np.zeros(VOCAB_SIZE)
            out[out_seq-1]=1

            y_in.append(in_seq)
            y_out.append(out)

    return X, y_in, y_out

In [4]:
X, y_in, y_out = generator(images_features, captions_dict)
len(X), len(y_in), len(y_out)

(105000, 105000, 105000)

In [5]:
X = np.array(X)
y_in = np.array(y_in, dtype='float64')
y_out = np.array(y_out, dtype='float64')
X.shape, y_in.shape, y_out.shape

((105000, 2048), (105000, 37), (105000, 2729))

In [ ]:
np.save('X.npy', X)
np.save('y_in.npy', y_in)
np.save('y_out.npy', y_out)

In [3]:
vocabulary=np.load('./vocabulary (1).npy')

In [4]:
X = np.load('./X.npy')
y_in = np.load('./y_in.npy')
y_out = np.load('./y_out.npy')

# model structure

In [6]:

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import add
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model

In [9]:
embedding_size = 64
max_len = 37
vocab_size = len(vocabulary)

image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))
image_model.add(Dropout(0.5))
image_model.summary()

# Define LSTM model
language_model = Sequential()
language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))
language_model.add(Dropout(0.5))
language_model.summary()

conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)

# model.load_weights("../input/model_weights.h5")
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                131136    
                                                                 
 repeat_vector (RepeatVecto  (None, 37, 64)            0         
 r)                                                              
                                                                 
 dropout (Dropout)           (None, 37, 64)            0         
                                                                 
Total params: 131136 (512.25 KB)
Trainable params: 131136 (512.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          17465

# loading  the trained model for training if trained before

In [7]:
from keras.models import load_model

# Load the model
model = load_model("trained_lstm_new.h5")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 embedding_input (InputLaye  [(None, None)]               0         []                            
 r)                                                                                               
                                                                                                  
 dense_input (InputLayer)    [(None, 2048)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 64)             174656    ['embedding_input[0][0]']     
                                                                                                  
 dense (Dense)               (None, 64)                   131136    ['dense_input[0][0]']     

# training the model


In [8]:
model.fit([X, y_in], y_out, batch_size=37, epochs=20)

Epoch 1/20
2838/2838 [==============================] - 58s 18ms/step - loss: 1.4544 - accuracy: 0.7629
Epoch 2/20
2838/2838 [==============================] - 44s 15ms/step - loss: 1.4485 - accuracy: 0.7637
Epoch 3/20
2838/2838 [==============================] - 44s 15ms/step - loss: 1.4450 - accuracy: 0.7648
Epoch 4/20
2838/2838 [==============================] - 44s 16ms/step - loss: 1.4421 - accuracy: 0.7650
Epoch 5/20
2838/2838 [==============================] - 43s 15ms/step - loss: 1.4373 - accuracy: 0.7653
Epoch 6/20
2838/2838 [==============================] - 44s 15ms/step - loss: 1.4295 - accuracy: 0.7662
Epoch 7/20
2838/2838 [==============================] - 44s 15ms/step - loss: 1.4268 - accuracy: 0.7670
Epoch 8/20
2838/2838 [==============================] - 43s 15ms/step - loss: 1.4273 - accuracy: 0.7671
Epoch 9/20
2838/2838 [==============================] - 44s 16ms/step - loss: 1.4339 - accuracy: 0.7671
Epoch 10/20
2838/2838 [==============================] - 43s 15m

# saving the trained model


In [9]:
# Save the model to the local file system
model.save("trained_lstm_new.h5")
# Download the saved model file
from google.colab import files
files.download("trained_lstm_new.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>